In [1]:
import os

import numpy as np
import math
import uproot as uproot
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.colors
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import ticker
from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)
from matplotlib import gridspec
import dunestyle.matplotlib as dunestyle

import scipy.stats as stats
from scipy.stats import norm
from scipy.stats import median_abs_deviation
from scipy.interpolate import CubicSpline
from scipy.optimize import curve_fit
import scipy.linalg as la
import scipy.optimize as opt
from scipy.optimize import Bounds, LinearConstraint
from scipy.stats import chisquare

import pandas_helpers as ph
import cc2p_reco_var as cc2preco
import stat_helpers as sh

np.seterr(divide='ignore', invalid='ignore', over='ignore')

DUNE plot style enabled


{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
## -- for MC spring 25, Moon processed CAF without SCE correction
#### ==== but measured c_cal of the sample is 2.019e-2

## Afro
mc_file = "/exp/sbnd/data/users/apapadop/dfs/v10_06_00_02/mc_MCP2025B_5e18_02_prodgenie_corsika_proton_rockbox_sbnd_CV_caf_flat_caf_sbnd_100files.df"
## Sungbin
#mc_file = "/exp/sbnd/data/users/apapadop/dfs/v10_06_00_02/2025B_final_valid_v_10_06_00_02_mc_bnb_cosmics_pandora.df"
mc_hdr_df = pd.read_hdf(mc_file, key='hdr')
mc_pot_df = pd.read_hdf(mc_file, key='pot')
mc_reco_df_ = pd.read_hdf(mc_file, key='evt')
mc_nudf_ = pd.read_hdf(mc_file, key='mcnu')
mc_primdf = pd.read_hdf(mc_file, key='mcprim')

In [3]:
#print(mc_primdf.rec.mc.nu.prim.pdg)
mask = np.abs(mc_primdf.rec.mc.nu.prim.pdg) == 321
mc_nu_kaon_df = mc_primdf[mask]
indices = mc_nu_kaon_df.index
print(indices)

print(len(mc_primdf))

MultiIndex([( 2,  3, 1, 2),
            (13, 15, 1, 2),
            (13, 16, 0, 3),
            (18, 12, 1, 3),
            (37, 26, 0, 2),
            (41,  9, 1, 4),
            (41,  9, 1, 5),
            (48,  9, 0, 1),
            (48,  9, 0, 4),
            (54,  4, 1, 2),
            (64, 13, 1, 2),
            (71,  2, 1, 3),
            (83,  0, 1, 3),
            (87, 18, 0, 2),
            (92,  8, 0, 3),
            (92,  8, 0, 4),
            (96, 20, 1, 2),
            (98,  4, 0, 2)],
           names=['__ntuple', 'entry', 'rec.mc.nu..index', 'rec.mc.nu.prim..index'])
17395


In [4]:
#print(mc_hdr_df.keys())
#print(len(mc_hdr_df))

mc_nu_kaon_df = ph.multicol_merge(mc_nu_kaon_df.reset_index(), mc_hdr_df.reset_index(),
        left_on=[('__ntuple', '', ''), ('entry', '', '')],
        right_on=[('__ntuple', '', ''), ('entry', '', '')],
        how='left'
    )

/exp/sbnd/app/users/apapadop/cafpyana/analysis_village/examples/cc2p_analysis/pandas_helpers.py:85: PerformanceWarning: indexing past lexsort depth may impact performance.
  return lhs.merge(rhs, **panda_kwargs)


In [5]:
print(mc_nu_kaon_df.rec.keys())

MultiIndex([('mc', 'nu', 'prim',   'genE',  ''),
            ('mc', 'nu', 'prim', 'length',  ''),
            ('mc', 'nu', 'prim',    'pdg',  ''),
            ('mc', 'nu', 'prim',   'genp', 'x'),
            ('mc', 'nu', 'prim',   'genp', 'y'),
            ('mc', 'nu', 'prim',   'genp', 'z'),
            ('mc', 'nu', 'prim',  'start', 'x'),
            ('mc', 'nu', 'prim',  'start', 'y'),
            ('mc', 'nu', 'prim',  'start', 'z'),
            ('mc', 'nu', 'prim',    'end', 'x'),
            ('mc', 'nu', 'prim',    'end', 'y'),
            ('mc', 'nu', 'prim',    'end', 'z')],
           )
